# Создание финальной разметки

In [3]:
import os
import tqdm
import pandas as pd
import re
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split

In [4]:
path_to_books = '/home/niknikiforov/RuPunctNet/data/prepared_data/books'

book_file_names = [i for i in os.listdir(path_to_books) if i != '.DS_Store']
len(book_file_names)

16

In [5]:
book_file_names.remove('bytie_i_vremya.txt')
book_file_names.remove('tak_govoril_zaratustra.txt')
book_file_names.remove('starik_i_more.txt')

In [6]:
len(book_file_names)

13

In [7]:
# в филосовских текстах присуствует зашумление текста из-за битый формул, второго языка и тд
book_file_names

['belye_nochi.txt',
 'po.txt',
 'mir_v_orehovoy-skorlupke.txt',
 'detstvo_tolstoi.txt',
 'voina_mir12.txt',
 'prestup_i_nakaz.txt',
 'chekhov.txt',
 'idiot.txt',
 'oblomov.txt',
 'voina_mir3.txt',
 'voina_mir4.txt',
 'master_margarita.txt',
 'anna_karenina.txt']

In [8]:
all_books_texts = []

for book_name in tqdm.tqdm(book_file_names): 
    with open(path_to_books + "/" + book_name, encoding='utf-8') as file:
        for line in file:
            line = line[:-1]
            
            # меняем тире
            line = re.sub('–', '—', line)
            
            # если реплика, то проверяем отстутсвие еще одного тире
            if (line[:2] == '— ') & (' — ' in line[2:]):
                continue
            if ('"' in line) & (':' in line):
                continue
                
            elif line[:2] == '— ':
                line = line[2:]
                
            # меняем доветочие и  троеточие
            line = re.sub('\?..', '?', line)
            line = re.sub('\!..', '!', line)
            line = re.sub('…', '...', line)
            
            # убираем технические знаки для ссылок
            
            line = re.sub(r'{\d}', '', line)
            line = re.sub('{', '', line)
            line = re.sub('}', '', line)
            
            # убираем тире
            
            line = re.sub(r'\s-\s', '', line)
            
            # убираем ;, ""
            
            line = re.sub(';', '.', line)
            line = re.sub('"', '', line)


            all_books_texts.append(line)
            
len(all_books_texts)

100%|███████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 25.12it/s]


26474

In [9]:
def clean_text_special_symbols(line):
    line = re.sub('– ', '', line)
    line = re.sub('— ', '', line)
    line = re.sub('\(', '', line)
    line = re.sub('\)', '', line)
    line = re.sub('"', '', line)
    line = line.lower()
    line = re.sub('\s+', ' ', line)
    
    return [token for token in line.split(' ') if token != '']


In [10]:
import string

In [11]:
def clear_punct_lower(line):
    line = line.lower()
    line = re.sub('–', '', line)
    line = re.sub('—', '', line).strip()
    
    # удаляем всю пунктуацию
    line = line.translate(str.maketrans('', '', string.punctuation))
    return line

In [12]:
def create_labels(token_list):
    
    labels_puncts = ['.', ',', '?', '!', ';', '...', ':']
    labels = []
    for token in token_list:
        if (len(token) > 3) & (token[-3:] == '...'):
            labels.append('...')
        elif token[-1] in labels_puncts:
            labels.append(token[-1])
        else:
            labels.append('o')
    
    return labels

In [13]:
def get_markup(df_text):
    '''
    Функция, принимающая на вход датафрейм с колонкой текстов (text) и
    возвращающая датафрейм с соответствующими метками.
    1) Список очищенных токенов для обучения
    2) Список токенов приведенных к нижнему регистру для будущей модели
    2) Метки основных знаков препинания:
    ['.', ',', '?', '!', ';', '...', ':', 'o']. (o = other)
    
    
    Таким образом, для каждого текста будет возвращаться 4 списка
    '''
    df_text['tokens'] = df_text['text'].apply(clean_text_special_symbols)
    df_text['clear_punct_lower'] = df_text['text'].apply(clear_punct_lower)
    
    labels_puncts = ['.', ',', '?', '!', ';', '...', ':']
    df_text['labels'] = df_text['tokens'].apply(create_labels)
    
    
    df_text['tokens'] = df_text['tokens'].apply(lambda x: [re.sub("[^\w\s]", '', y) for y in x])
    
    return df_text


In [38]:
books_markup = pd.DataFrame()
books_markup['text'] = all_books_texts
books_markup = get_markup(books_markup)
books_markup

,text,tokens,clear_punct_lower,labels
0,"Итак, вы понимаете, читатель, каким образом я ...","[итак, вы, понимаете, читатель, каким, образом...",итак вы понимаете читатель каким образом я зна...,"[,, o, ,, ,, o, o, o, o, o, o, .]"
1,"Я ходил много и долго, так что уже совсем успе...","[я, ходил, много, и, долго, так, что, уже, сов...",я ходил много и долго так что уже совсем успел...,"[o, o, o, o, ,, o, o, o, o, ,, o, o, ,, ,, o, ..."
2,А все-таки моя ночь была лучше дня!от как это ...,"[а, всетаки, моя, ночь, была, лучше, дняот, ка...",а всетаки моя ночь была лучше дняот как это было,"[o, o, o, o, o, o, o, o, o, .]"
3,"Я пришел назад в город очень поздно, и уже про...","[я, пришел, назад, в, город, очень, поздно, и,...",я пришел назад в город очень поздно и уже проб...,"[o, o, o, o, o, o, ,, o, o, o, o, ,, o, o, o, ..."
4,"По той стороне тротуара, недалеко от моей незн...","[по, той, стороне, тротуара, недалеко, от, мое...",по той стороне тротуара недалеко от моей незна...,"[o, o, o, ,, o, o, o, ,, o, o, o, o, ,, o, ,, ..."
...,...,...,...,...
26469,И разве астрономы могли бы понять и вычислить ...,"[и, разве, астрономы, могли, бы, понять, и, вы...",и разве астрономы могли бы понять и вычислить ...,"[o, o, o, o, o, o, o, o, ,, o, o, o, o, o, o, ..."
26470,"Но она все-таки не рассмотрела бы его лица, ес...","[но, она, всетаки, не, рассмотрела, бы, его, л...",но она всетаки не рассмотрела бы его лица если...,"[o, o, o, o, o, o, o, ,, o, o, o, ,, o, ,, o, ..."
26471,"Она понимает, — думал он, — она знает, о чем я...","[она, понимает, думал, он, она, знает, о, чем,...",она понимает думал он она знает о чем я дума...,"[o, ,, o, ,, o, ,, o, o, o, ., o, o, o, ,, o, ..."
26472,"Нет, не надо говорить, — подумал он, когда она...","[нет, не, надо, говорить, подумал, он, когда, ...",нет не надо говорить подумал он когда она про...,"[,, o, o, ,, o, ,, o, o, o, o, ., o, ,, o, o, ..."


In [25]:
books_markup.to_csv("new_books_prepared.csv")

In [16]:
markup_stats = Counter()

for row in tqdm.tqdm(books_markup.values):
    markup_stats += Counter(row[3])

markup_stats 

100%|█████████████████████████████████████████████████████████████████████████| 26474/26474 [00:00<00:00, 186406.94it/s]


Counter({'o': 891801,
         ',': 150475,
         '.': 65898,
         '?': 3119,
         '!': 2328,
         ':': 2168,
         '...': 1889})

In [17]:
for key, value in markup_stats.items():
    print(f"Share of {key} = {np.round(value/(sum(markup_stats.values())-markup_stats['o']), 4)}")

Share of , = 0.6662
Share of o = 3.9482
Share of . = 0.2917
Share of ... = 0.0084
Share of ! = 0.0103
Share of : = 0.0096
Share of ? = 0.0138


# Gigachat augmentation

In [20]:
from gigachat import GigaChat

In [21]:
giga = GigaChat(credentials=GIGACHAT_TOKEN,model="GigaChat-Pro", verify_ssl_certs=False)

In [26]:
responses = []

for line_index in tqdm.tqdm(range(books_markup.shape[0])):
    line = books_markup.text.iloc[line_index]
    try:
        req = 'Перефразируй текст и добавь в него эмоциональные знаки препинания ?, ! и ...: ' + line
        responses.append(giga.chat(req).choices[0].message.content)
    except:
        break

  1%|▍                                                                           | 137/26474 [04:47<15:21:04,  2.10s/it]


In [31]:
gig_markup = pd.DataFrame()
gig_markup['text'] = responses
gig_markup = get_markup(gig_markup)
gig_markup

,text,tokens,clear_punct_lower,labels
0,"И так, вы понимаете, читатель, каким образом я...","[и, так, вы, понимаете, читатель, каким, образ...",и так вы понимаете читатель каким образом я зн...,"[o, ,, o, ,, ,, o, o, o, o, o, o, !]"
1,"Я ходил много и долго, так что уже совсем успе...","[я, ходил, много, и, долго, так, что, уже, сов...",я ходил много и долго так что уже совсем успел...,"[o, o, o, o, ,, o, o, o, o, ,, o, o, ,, ,, o, ..."
2,А все-таки моя ночь была лучше дня! Вот как эт...,"[а, всетаки, моя, ночь, была, лучше, дня, вот,...",а всетаки моя ночь была лучше дня вот как это ...,"[o, o, o, o, o, o, !, o, o, o, :, o, o, ,, o, ..."
3,"Я пришел назад в город очень поздно, и уже про...","[я, пришел, назад, в, город, очень, поздно, и,...",я пришел назад в город очень поздно и уже проб...,"[o, o, o, o, o, o, ,, o, o, o, o, ,, o, o, o, ..."
4,"По той стороне тротуара, недалеко от моей незн...","[по, той, стороне, тротуара, недалеко, от, мое...",по той стороне тротуара недалеко от моей незна...,"[o, o, o, ,, o, o, o, ,, o, o, o, o, ,, o, ,, ..."
...,...,...,...,...
132,"Ах, да! И у нас тоже Фекла!","[ах, да, и, у, нас, тоже, фекла]",ах да и у нас тоже фекла,"[,, !, o, o, o, o, !]"
133,"Матрена добрая, только один недостаток: у ней ...","[матрена, добрая, только, один, недостаток, у,...",матрена добрая только один недостаток у ней не...,"[o, ,, o, o, :, o, o, o, ,, ,, o, o, ., o, o, !]"
134,Все равно! Они обе могут быть вместе. Только в...,"[все, равно, они, обе, могут, быть, вместе, то...",все равно они обе могут быть вместе только вы ...,"[o, !, o, o, o, o, ., o, o, o, o, o, !]"
135,"Ах, Настенька!","[ах, настенька]",ах настенька,"[,, !]"


In [37]:
books_markup.head(137)

,text,tokens,clear_punct_lower,labels
0,"Итак, вы понимаете, читатель, каким образом я ...","[итак, вы, понимаете, читатель, каким, образом...",итак вы понимаете читатель каким образом я зна...,"[,, o, ,, ,, o, o, o, o, o, o, .]"
1,"Я ходил много и долго, так что уже совсем успе...","[я, ходил, много, и, долго, так, что, уже, сов...",я ходил много и долго так что уже совсем успел...,"[o, o, o, o, ,, o, o, o, o, ,, o, o, ,, ,, o, ..."
2,А все-таки моя ночь была лучше дня!от как это ...,"[а, всетаки, моя, ночь, была, лучше, дняот, ка...",а всетаки моя ночь была лучше дняот как это было,"[o, o, o, o, o, o, o, o, o, .]"
3,"Я пришел назад в город очень поздно, и уже про...","[я, пришел, назад, в, город, очень, поздно, и,...",я пришел назад в город очень поздно и уже проб...,"[o, o, o, o, o, o, ,, o, o, o, o, ,, o, o, o, ..."
4,"По той стороне тротуара, недалеко от моей незн...","[по, той, стороне, тротуара, недалеко, от, мое...",по той стороне тротуара недалеко от моей незна...,"[o, o, o, ,, o, o, o, ,, o, o, o, o, ,, o, ,, ..."
...,...,...,...,...
132,"Ах, да и у нас тоже Фекла!","[ах, да, и, у, нас, тоже, фекла]",ах да и у нас тоже фекла,"[,, o, o, o, o, o, !]"
133,"Матрена добрая, только один недостаток: у ней ...","[матрена, добрая, только, один, недостаток, у,...",матрена добрая только один недостаток у ней не...,"[o, ,, o, o, :, o, o, o, ,, ,, o, o, ., o, o, !]"
134,Все равно. они обе могут быть вместе. только в...,"[все, равно, они, обе, могут, быть, вместе, то...",все равно они обе могут быть вместе только вы ...,"[o, ., o, o, o, o, ., o, o, o, o, o, .]"
135,"Ах, Настенька!","[ах, настенька]",ах настенька,"[,, !]"


In [35]:
markup_stats_giga = Counter()

for row in tqdm.tqdm(gig_markup.values):
    markup_stats_giga += Counter(row[3])

for key, value in markup_stats_giga.items():
    print(f"Share of {key} = {np.round(value/(sum(markup_stats_giga.values())-markup_stats_giga['o']), 4)}")

100%|█████████████████████████████████████████████████████████████████████████████| 137/137 [00:00<00:00, 180244.56it/s]

Share of o = 2.8183
Share of , = 0.5746
Share of ! = 0.1574
Share of . = 0.1452
Share of : = 0.0183
Share of ... = 0.066
Share of ? = 0.0386


In [40]:
markup_stats_giga = Counter()

for row in tqdm.tqdm(books_markup.values[:137]):
    markup_stats_giga += Counter(row[3])

for key, value in markup_stats_giga.items():
    print(f"Share of {key} = {np.round(value/(sum(markup_stats_giga.values())-markup_stats_giga['o']), 4)}")

100%|█████████████████████████████████████████████████████████████████████████████| 137/137 [00:00<00:00, 194390.95it/s]

Share of , = 0.6036
Share of o = 2.9599
Share of . = 0.2639
Share of ... = 0.0312
Share of ! = 0.0434
Share of : = 0.0178
Share of ? = 0.0401
